References: 

- https://www.youtube.com/watch?v=xT6JpDELKPg&list=WL&index=13

- https://docs.llamaindex.ai/en/latest/examples/query_engine/sec_tables/tesla_10q_table.html

In [1]:
# !pip install llama-index llama-hub unstructured pypdf --upgrade

### Extract Elements

In [2]:
# from pydantic import BaseModel
from unstructured.partition.html import partition_html
import pandas as pd

Load document from url and save locally

In [3]:
!wget "https://www.dropbox.com/scl/fi/mlaymdy1ni1ovyeykhhuk/tesla_2021_10k.htm?rlkey=qf9k4zn0ejrbm716j0gg7r802&dl=1" -O tesla_2021_10k.htm

--2023-11-12 19:07:12--  https://www.dropbox.com/scl/fi/mlaymdy1ni1ovyeykhhuk/tesla_2021_10k.htm?rlkey=qf9k4zn0ejrbm716j0gg7r802&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.13.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.13.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uce770ca79abe82c2400b7bbfa05.dl.dropboxusercontent.com/cd/0/inline/CHZi8cFpSXXAeJ50DZ6BNupY3aPw13W1kXgLaabIKmKrMsENMqqvHuaI8xdLW6PsE2sArQQ87QD1IsJuMiD2uEsQWl338VfWjF93ZKZq3dikjW3jhfjt-2xrBbvqCarvqTo/file?dl=1# [following]
--2023-11-12 19:07:13--  https://uce770ca79abe82c2400b7bbfa05.dl.dropboxusercontent.com/cd/0/inline/CHZi8cFpSXXAeJ50DZ6BNupY3aPw13W1kXgLaabIKmKrMsENMqqvHuaI8xdLW6PsE2sArQQ87QD1IsJuMiD2uEsQWl338VfWjF93ZKZq3dikjW3jhfjt-2xrBbvqCarvqTo/file?dl=1
Resolving uce770ca79abe82c2400b7bbfa05.dl.dropboxusercontent.com (uce770ca79abe82c2400b7bbfa05.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27d:10

Load from local machine

In [5]:
from llama_index.readers.file.flat_reader import FlatReader
from pathlib import Path

In [6]:
reader = FlatReader()
docs_2021 = reader.load_data(Path("tesla_2021_10k.htm"))

In [7]:
from llama_index.node_parser import (
                                     UnstructuredElementNodeParser,
                                    )

node_parser = UnstructuredElementNodeParser()

In [10]:
# !pip install python-dotenv

In [11]:
from dotenv import load_dotenv
load_dotenv("keys.env")  # should be in the same directory path as this notebook; env file needs to have key to use LLM API (OpenAI in this case)

True

In [12]:
import os
import pickle

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Extract nodes based on content/layout (e.g. text vs. tables) in the document. Note: This takes a few minutes to run.
if not os.path.exists("2021_nodes.pkl"):  # if not there already 
    raw_nodes_2021 = node_parser.get_nodes_from_documents(docs_2021)  # extract nodes; is done through LLM, under the hood
    pickle.dump(raw_nodes_2021, open("2021_nodes.pkl", "wb"))  # save locally
else:  # Load from local file, if nodes have been already extracted
    raw_nodes_2021 = pickle.load(open("2021_nodes.pkl", "rb"))

100%|██████████| 105/105 [10:45<00:00,  6.15s/it]


In [13]:
# Build hierarchy of nodes, on the basis of the doc layout (as understood by the LLM model)
base_nodes_2021, node_mappings_2021 = node_parser.get_base_nodes_and_mappings(
                                                                              raw_nodes_2021
                                                                             )

Let's see what info is captured in a specific node

In [15]:
from llama_index.schema import IndexNode

example_index_node = [b for b in base_nodes_2021 if isinstance(b, IndexNode)][20]

# Index Node
print(
      f"\n--------\n{example_index_node.get_content(metadata_mode='all')}\n--------\n"
     )

# Index Node ID
print(f"\n--------\nIndex ID: {example_index_node.index_id}\n--------\n")

# Referenced Table
print(
      f"\n--------\n{node_mappings_2021[example_index_node.index_id].get_content()}\n--------\n"
    )


--------
col_schema: Column: Type
Type: string
Summary: Type of net income (loss) per share calculation (basic or diluted)

Column: Amount
Type: string
Summary: Net income (loss) per share amount

Column: Year 1
Type: string
Summary: Net income (loss) per share for year 1

Column: Year 2
Type: string
Summary: Net income (loss) per share for year 2

Column: Year 3
Type: string
Summary: Net income (loss) per share for year 3

Summary of net income (loss) per share of common stock attributable to common stockholders
--------


--------
Index ID: id_617_table
--------


--------
                                                                                                                                                                                                                                             
0                                                                                                                       Year Ended December 31,                                    

### Build Recursive Retriever

In [16]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.llms import OpenAI

In [17]:
llm = OpenAI(model="gpt-4-1106-preview")
service_context = ServiceContext.from_defaults(llm=llm)

Build a **baseline retriever** that simply does a top-k lookup over the raw document. i.e. one-size-fits-all approach irrespective of doc layout

In [18]:
# Construct top-level vector index + query engine
vector_index = VectorStoreIndex(base_nodes_2021,  # Note: It's NOT using node_mappings learnt above i.e. It ignores the hierarchy/layout of content
                                service_context=service_context)
vector_retriever = vector_index.as_retriever(similarity_top_k=1)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=1)

Build a **retriever which uses the hierarchy (node-mappings)** info learnt above and (hopefully) performs better.

The top-level vector index does top-k lookup over a set of **Nodes**.

A `RecursiveRetriever` object will recursively retrieve/query nodes. We then put this in our `RetrieverQueryEngine` along with a `ResponseSynthesizer` to synthesize a response.

We pass in mappings from id to retriever and id to query engine. We then pass in a root id representing the retriever we query first.

In [19]:
from llama_index.retrievers import RecursiveRetriever

recursive_retriever = RecursiveRetriever(
                                         "vector",
                                         retriever_dict={"vector": vector_retriever},
                                         node_dict=node_mappings_2021,  # use the layout/hierarchy of content
                                         verbose=True,
                                        )

In [20]:
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

### Compare response of the (hieararchical node-based) RecursiveRetriever RAG w.r.t. that of the naive RAG

Example 1

In [27]:
# node-based recursive retriever
response = query_engine.query("What was the Net cash provided by operating activities in 2021?")
print(str(response))

Retrieving with query id None: What was the Net cash provided by operating activities in 2021?
Retrieved node with id, entering: id_558_table
Retrieving with query id id_558_table: What was the Net cash provided by operating activities in 2021?
The Net cash provided by operating activities in 2021 was $11,497 million.


In [28]:
# naive RAG retriever
response = vector_query_engine.query("What was the Net cash provided by operating activities in 2021?")
print(str(response))

The summary of net cash flows provided does not include specific figures for the Net cash provided by operating activities in 2021. To answer the query, the actual financial data for the year ended December 31, 2021, would be required.


Example 2

In [30]:
# node-based recursive retriever
query_engine.query("What is the difference in total lease expense between the years 2020 and 2021?")

Retrieving with query id None: What is the difference in total lease expense between the years 2020 and 2021?
Retrieved node with id, entering: id_1392_table
Retrieving with query id id_1392_table: What is the difference in total lease expense between the years 2020 and 2021?


Response(response='The difference in total lease expense between the years 2020 and 2021 is $232.', source_nodes=[NodeWithScore(node=TextNode(id_='id_1392_table', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='41763380abca0099d6ea17975cca278875a745650d63677ada360350ca5d54f0', text='                                                                                                                                 \n0                                          Year Ended December 31,                                                               \n1                                                             2021                 2020                2019                      \n2          Operating lease expense:                                                                                              \n3       Operating lease expense (1)                              $      627                $    451                $    426

In [31]:
# naive RAG retriever
vector_query_engine.query("What is the difference in total lease expense between the years 2020 and 2021?")

Response(response='The specific difference in total lease expense between the years 2020 and 2021 is not provided in the context information. To determine the difference, the actual total lease expense amounts for each year would be required.', source_nodes=[NodeWithScore(node=IndexNode(id_='id_1392_table_ref', embedding=None, metadata={'col_schema': 'Column: Operating lease expense (1)\nType: Currency\nSummary: Operating lease expenses\n\nColumn: Amortization of leased assets\nType: Currency\nSummary: Amortization of leased assets for finance leases\n\nColumn: Interest on lease liabilities\nType: Currency\nSummary: Interest expenses on lease liabilities for finance leases\n\nColumn: Total finance lease expense\nType: Currency\nSummary: Total expense for finance leases\n\nColumn: Total lease expense\nType: Currency\nSummary: Total lease expenses'}, excluded_embed_metadata_keys=['col_schema'], excluded_llm_metadata_keys=[], relationships={}, hash='cfc090358cca853a84c36579069557573de47f1

Example 3

In [40]:
# node-based recursive retriever
query_engine.query("What are the total deferred tax taxes in 2020?")

Retrieving with query id None: What are the total deferred tax taxes in 2020?
Retrieved node with id, entering: id_1564_table
Retrieving with query id id_1564_table: What are the total deferred tax taxes in 2020?


Response(response='The total deferred tax liabilities in 2020 were $2,102.', source_nodes=[NodeWithScore(node=TextNode(id_='id_1564_table', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='f63deb35a15a6d6606a81bf1f7798bedcc3380e07023c976002ca485769f675c', text='                                                                                                                                                                            \n0                                                                                   December 31,                             December 31,                                   \n1                                                                             2021                                     2020                                                 \n2                                              Deferred tax assets:                                                                                 

In [41]:
# naive RAG retriever
vector_query_engine.query("What are the total deferred tax taxes in 2020?")

Response(response='To calculate the total deferred tax taxes for 2020, you would need to have the specific numeric values for both the deferred tax assets and the deferred tax liabilities for that year. Once you have those values, you would typically subtract the deferred tax liabilities from the deferred tax assets to determine the net deferred tax amount. If the information for 2020 is not provided, it is not possible to calculate the total deferred tax taxes for that year.', source_nodes=[NodeWithScore(node=IndexNode(id_='id_1564_table_ref', embedding=None, metadata={'col_schema': 'Column: Deferred tax assets\nType: numeric\nSummary: Amount of deferred tax assets\n\nColumn: Deferred tax liabilities\nType: numeric\nSummary: Amount of deferred tax liabilities'}, excluded_embed_metadata_keys=['col_schema'], excluded_llm_metadata_keys=[], relationships={}, hash='01b987e84661e7d9dfabeb02b1bdb9be4d134f2e30139a6865cb8290bc75c733', text='Summary of deferred tax assets and liabilities', star